In [1]:
import syft as sy

In [3]:
scc = sy.login(url="http://172.20.58.117:8081/", email="info@openmined.org", password="changethis")


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to http://172.20.58.117:8081/... done! 	 Logging into scc... done!


In [4]:
scc.users


,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,16cc388b1e7daac72f8fadd587788f21589add3efc827d...,Owner,None,None,None,None,2021-12-31 07:52:21.522472,5.55


In [5]:
scc.users.create(
    **{
        "name": "zetyun_psi",
        "email": "sample@zetyun.com",
        "password": "zetyun",
        "budget":200
    }
)

In [6]:
scc.users

,id,email,name,budget,verify_key,role,added_by,website,institution,daa_pdf,created_at,budget_spent
0,1,info@openmined.org,Jane Doe,5.55,16cc388b1e7daac72f8fadd587788f21589add3efc827d...,Owner,None,None,None,NaN,2021-12-31 07:52:21.522472,5.55
1,2,sample@zetyun.com,zetyun_psi,200.00,516003db543d69e8aa53649779315badb720d5e0d8c9c2...,Data Scientist,Jane Doe,,,1.0,2021-12-31 08:37:28.905109,200.00


In [7]:
import load

ids = load.load_column("dataset/scc.csv", "id", "str")


------------------------dataset/scc.csv------------------------
                                 id name
0  60dcd6597ac34d97aa2fc913b95009c4  李八搜
1  b01d7b6de97b43eb82ee759da3397852  孙一捷
2  c08f401620a744c0abe9417c7c15495e  黄六引
3  01e95a079e9040db8be63d2cfea342e4  钱三文
4  f586a37343b44a5a9a50bb0c8c4d23e1  钱四力
------------------------id------------------------
array([ 450007691,  901885509, 1372689838,  466742559, -736305036])


In [8]:
scc.load_dataset(
    assets={
        "scc_id": sy.Tensor(ids).private(min_val=float(ids.min()), max_val=float(ids.max()), entities="老赖数据:scc:id"),
    },
    name="scc",
    description="老赖数据:scc"
)

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [9]:
scc.datasets

Idx,Name,Description,Assets,Id
[0],scc,老赖数据:scc,"[""scc_id""] -> Tensor",493a00b3-e61a-433e-80fd-ecab9e055d3b


In [10]:
id_ptr = scc.datasets[0]["scc_id"]
print(id_ptr)
id_ptr.request(reason="需要获取准确数据, 准备求交")
scc.requests

array([-1659229690, -1219033503, -1137083311, ...,  1105562285,
         512543903,  -718007060])

 (The data printed above is synthetic - it's an imitation of the real data.)


,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,pending,"需要获取准确数据, 准备求交",<UID: a9812e69e6d944188d2efdae578a0e4f>,<UID: 75323e0cb6ed46479ab45e0198ba5b2e>,[#scc_id],None,None


In [11]:
scc.requests[0].accept()

In [12]:
scc_id_list = id_ptr.get(delete_obj=False)


In [13]:
scc.store

,ID,Tags,Description,object_type
0,<UID: 75323e0cb6ed46479ab45e0198ba5b2e>,[#scc_id],,<class 'syft.core.tensor.tensor.Tensor'>
1,<UID: c5665845d6bd4ae2b526d9bc5db95353>,[client_items_len],"求交初始化: 请服务器设置好, fpr=1e-6, reveal_intersection=...",<class 'syft.lib.python.Int'>


In [14]:
scc.store[1].request(reason="获取client_items_len")

In [15]:
scc.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,accepted,"需要获取准确数据, 准备求交",<UID: a9812e69e6d944188d2efdae578a0e4f>,<UID: 75323e0cb6ed46479ab45e0198ba5b2e>,[#scc_id],None,None
1,Jane Doe,info@openmined.org,Owner,DATA,pending,获取client_items_len,<UID: 4a84e7df0b054aa8934ed7d96af9f06e>,<UID: c5665845d6bd4ae2b526d9bc5db95353>,[client_items_len],None,None


In [16]:
scc.requests[1].accept()

In [17]:
client_items_len = scc.store[1].get(delete_obj=False)

In [18]:
import openmined_psi as psi

server = psi.server.CreateWithNewKey(True)

In [19]:
server_items = [str(i) for i in scc_id_list.child.child.tolist()]
# print(server_items)
print(client_items_len)

10000


In [20]:

setup = server.CreateSetupMessage(1e-6, client_items_len, server_items)


In [21]:
setupMsg = setup.SerializeToString()
wrapper = sy.lib.python.List()
wrapper.append(setupMsg)
wrapper.send(scc, pointable=True, tags=["setup"], description="服务器设置好了")


<ListPointer -> scc:bf6dbcd68d8942a1b4626d67bdfaa452>

In [22]:
scc.store

,ID,Tags,Description,object_type
0,<UID: 75323e0cb6ed46479ab45e0198ba5b2e>,[#scc_id],,<class 'syft.core.tensor.tensor.Tensor'>
1,<UID: c5665845d6bd4ae2b526d9bc5db95353>,[client_items_len],"求交初始化: 请服务器设置好, fpr=1e-6, reveal_intersection=...",<class 'syft.lib.python.Int'>
2,<UID: bf6dbcd68d8942a1b4626d67bdfaa452>,[setup],服务器设置好了,<class 'syft.lib.python.list.List'>


In [23]:
[(x.user_name, x.id, x.request_description) for x in scc.requests.requests if x.request_description != '' and x.status != "accepted"]

[('zetyun_psi', <UID: dac05517400d4dc489450db17201d135>, '客户端需要初始设置')]

In [24]:
[x for x in scc.requests.requests if x.request_description != '' and x.status != "accepted"][0].grant()

In [25]:
scc.store

,ID,Tags,Description,object_type
0,<UID: 75323e0cb6ed46479ab45e0198ba5b2e>,[#scc_id],,<class 'syft.core.tensor.tensor.Tensor'>
1,<UID: c5665845d6bd4ae2b526d9bc5db95353>,[client_items_len],"求交初始化: 请服务器设置好, fpr=1e-6, reveal_intersection=...",<class 'syft.lib.python.Int'>
2,<UID: bf6dbcd68d8942a1b4626d67bdfaa452>,[setup],服务器设置好了,<class 'syft.lib.python.list.List'>
3,<UID: af32fc69463b4962892a9fd773480fd1>,[request],求交request,<class 'syft.lib.python.list.List'>


In [26]:
scc.store[3].request(reason="获取客户端request")

In [27]:
[(x.user_name, x.id, x.request_description) for x in scc.requests.requests if x.request_description != '' and x.status != "accepted"]

[('Jane Doe', <UID: 711f122a527a4929b0fa1c6982c45bd8>, '获取客户端request')]

In [28]:
# scc.requests[str(uuid.UUID("13407e2317964d81aa415c6fd5bbdb28"))].accept()
[x for x in scc.requests.requests if x.request_description != '' and x.status != "accepted"][0].grant()

In [29]:
msg = scc.store[3].get(delete_obj=False)

In [30]:
request = psi.Request()
request.ParseFromString(msg[0])


350002

In [31]:
response = server.ProcessRequest(request)

In [32]:
repMsg = response.SerializeToString()
wrapper = sy.lib.python.List()
wrapper.append(repMsg)
wrapper.send(scc, pointable=True, tags=["response"], description="求交结果计算好了")

<ListPointer -> scc:25063f8efbd64d0e863f3329b0c38e39>

In [33]:
[(x.user_name, x.id, x.request_description) for x in scc.requests.requests if x.request_description != '' and x.status != "accepted"]

[('zetyun_psi', <UID: 08d91befe17e4bdd9d29110d4c5e3e5f>, '客户端要看结果')]

In [34]:
[x for x in scc.requests.requests if x.request_description != '' and x.status != "accepted"][0].grant()